In [1]:
import requests
from bs4 import BeautifulSoup

r = requests.get("https://pythonizing.github.io/data/real-estate/rock-springs-wy/LCWYROCKSPRINGS/")
c = r.content

soup = BeautifulSoup(c, "html.parser")

all_properties = soup.find_all("div", {"class": "propertyRow"})

In [2]:
all_properties[0].find("h4", {"class": "propPrice"}).text.replace("\n", "").replace(" ", "")

'$725,000'

In [3]:
properties = []

for property in all_properties:
    prop_dict = {}
    
    prop_dict["Address"] = property.find_all("span", {"class": "propAddressCollapse"})[0].text
    
    try:
        prop_dict["Area"] = property.find("span", {"class": "infoSqFt"}).find("b").text
    except:
        prop_dict["Area"] = None    
    
    try:
        prop_dict["Beds"] = property.find("span", {"class": "infoBed"}).find("b").text
    except:
        prop_dict["Beds"] = None
        
    try:
        prop_dict["Full Baths"] = property.find("span", {"class": "infoValueFullBath"}).find("b").text
    except:
        prop_dict["Full Baths"] = None
        
    try:
        prop_dict["Half Baths"] = property.find("span", {"class": "infoValueHalfBath"}).find("b").text
    except:
        prop_dict["Half Baths"] = None
        
    prop_dict["Locality"] = property.find_all("span", {"class": "propAddressCollapse"})[1].text
        
    for column_group in property.find_all("div", {"class": "columnGroup"}):
        #print(column_group)
        for feature_group, feature_name in zip(column_group.find_all("span", {"class": "featureGroup"}), column_group.find_all("span", {"class": "featureName"})):
            if "Lot Size" in feature_group.text:
                prop_dict["Lot Size"] = feature_name.text
                
    prop_dict["Price"] = property.find("h4", {"class": "propPrice"}).text.replace("\n", "").replace(" ", "")
                
    properties.append(prop_dict)

In [4]:
import pandas
df = pandas.DataFrame(properties)

In [5]:
df

,Address,Area,Beds,Full Baths,Half Baths,Locality,Price,Lot Size
0,0 Gateway,None,None,None,None,"Rock Springs, WY 82901","$725,000",NaN
1,1003 Winchester Blvd.,None,4,4,None,"Rock Springs, WY 82901","$452,900",0.21 Acres
2,600 Talladega,"3,154",5,3,None,"Rock Springs, WY 82901","$396,900",NaN
3,3239 Spearhead Way,"3,076",4,3,1,"Rock Springs, WY 82901","$389,900","Under 1/2 Acre,"
4,522 Emerald Street,"1,172",3,3,None,"Rock Springs, WY 82901","$254,000","Under 1/2 Acre,"
5,1302 Veteran's Drive,"1,932",4,2,None,"Rock Springs, WY 82901","$252,900",0.27 Acres
6,1021 Cypress Cir,"1,676",4,3,None,"Rock Springs, WY 82901","$210,000","Under 1/2 Acre,"
7,913 Madison Dr,"1,344",3,2,None,"Rock Springs, WY 82901","$209,000","Under 1/2 Acre,"
8,1344 Teton Street,"1,920",3,2,None,"Rock Springs, WY 82901","$199,900","Under 1/2 Acre,"
9,4 Minnies Lane,"1,664",3,2,None,"Rock Springs, WY 82901","$196,900",2.02 Acres


In [6]:
df.to_csv("properties.csv")